In [324]:
f_path = 'C:\\Users\\Ali Salem\\Desktop\\App_Update\\static/files\\parameters.xlsx'
slides_name = ['Trade Margin Table By Sector', 'Trade Margin Table By Segment', 'Trade Margin Table By Subsegment', 'Trade Margin Table By Subcategory']

### Libraries & reading pickle files

In [325]:
import os
%run "{os.path.dirname(os.getcwd())}\general_functions\generalFunctions.ipynb" #container

%run "{os.getcwd()}\Financials Replacement Function.ipynb" #container

In [326]:
filename = 'parameters.xlsx'

# Get the current working directory
current_dir = os.path.dirname(os.getcwd())

# Construct the full path to the file
f_path = os.path.join(current_dir, filename)
print(f_path)
#xls = pd.ExcelFile(f_path)
parm = pd.read_excel(f_path, sheet_name='Financials')
fields = dict(zip(parm['Field'],parm['Value']))


c:\Users\Ali Salem\Desktop\App_Update\parameters.xlsx


In [327]:
themePath = os.getcwd()+"\Theme1.thmx"

In [328]:
server = "powerbi://api.powerbi.com/v1.0/myorg/"+ fields['server']
dataset_name = fields['f_name']
f_name = os.getcwd()+"/"+fields['f_name']+".xlsx"

client_manuf = list(set(fields['client_manuf'].split(','))-set(['']))
client_brands = list(set(fields['client_brands'].split(','))-set(['']))

ManufOrTopC = fields['ManufOrTopC']
BrandOrTopB = fields['BrandOrTopB']

decimals = fields['decimals']
sign = fields['sign']
currency = fields['currency']
currency = ' '+ currency if sign.lower() == 'after' else  currency + ' ' 

prodORitem=fields["prodORitem"]
categories = list(set(fields['categories'].split(','))-set(['']))
sectors=list(set(fields['sectors'].split(','))-set(['']))
segments=list(set(fields['segments'].split(','))-set(['']))
subsegments=list(set(fields['subsegments'].split(','))-set(['']))
subcategories=list(set(fields['subcategories'].split(','))-set(['']))

national=fields['national']
customareas=fields['customareas']
print(fields['customareas'])
if pd.isna(customareas):
    customareas = ''
areas = list(set(fields['areas'].split(','))-set(['']))+[customareas]
areas = [a for a in areas if a != '']

regions_RET = list(set(fields['regions_RET'].split(','))-set(['']))
channels_RET = list(set(fields['channels_RET'].split(','))-set(['']))
market_RET=list(set(fields['market_RET'].split(','))-set(['']))

regions_CHAN=list(set(fields['regions_CHAN'].split(','))-set(['']))
channels_CHAN=list(set(fields['channels_CHAN'].split(','))-set(['']))
market_CHAN=list(set(fields['market_CHAN'].split(','))-set(['']))

regions_CUST=list(set(fields['regions_CUST'].split(','))-set(['']))
channels_CUST=list(set(fields['channels_CUST'].split(','))-set(['']))
market_CUST=list(set(fields['market_CUST'].split(','))-set(['']))

data_source=fields['data_source']

years=list(set(fields['years'].split(','))-set(['']))
end_date=fields['end_date']

brands_only=fields['brands_only']
National=["NATIONAL"]if national else []
percent = fields['percent']
percentstr= fields['percentstr']


client_brands,ManufOrTopC,decimals,sign,currency,prodORitem,categories

REGIONS


(['Schick', 'Equate', 'Cremo'],
 'Top Companies',
 2,
 'Before',
 '$ ',
 'SKU',
 ['Manual Shave Men'])

In [329]:
regions = regions_RET + regions_CHAN + regions_CUST
channels = channels_RET + channels_CHAN + channels_CUST
markets = market_RET + market_CHAN + market_CUST

entity_hierarchy = [
    ("Area",National),
    ("Region", regions),
    ("Channel", channels),
    ("Market", markets)
]
hierarchy_levels = [
    ("Category", categories),
    ("Sector", sectors),
    ("Segment", segments),
    ("SubSegment", subsegments),
    ("SubCategory", subcategories)
]

direct_parent = {"Sector":list(zip(["Category"]*len(categories),categories))
                ,"Segment":list(zip(["Sector"]*len(sectors),sectors))
                #,"SubSegment":list(zip(["Segment"]*len(sectors),sectors))
                #,"SubCategory":list(zip(["Segment"]*len(segments),segments))
                }


In [330]:
# client_manuf = ["Pbg"]
# client_brands = ["Schick","Equate","Cremo"]
 
# decimals = 2
# sign = "Before"
# currency = '$'
# currency = ' '+ currency if sign.lower() == 'after' else  currency + ' '
 
# prodORitem = "SKU"
# categories = ["Manual Shave Men"]
# sectors = ["System","Disposables"]
# segments = ["Razors","Refills","Disposables"]
# subsegments= []
# subcategories= []

# national = False
# customareas= "REGIONS"
# areas = ["RETAILER", f"{customareas}"]

# regions_RET  = ["Walmart"]
# channels_RET = []
# market_RET = ["Walmart Div1 Corp", "Walmart Nm Corp","Walmart Sc Corp"]

# regions_CHAN = []
# channels_CHAN = []
# market_CHAN = []
 
# regions_CUST = []
# channels_CUST = []
# market_CUST = ["Walmart East", "Walmart North","Walmart Southeast","Walmart Southwest","Walmart West"]


# data_source = "DATA SOURCE: Client P&L"

# end_date = "2025-04-01"

# OpenEditData=True
# ManufOrTopC ="Top Companies"
# BrandOrTopB="Top Brands"

# percent = 1000000
# percentstr="'000 000"
# ### OpenEditData is a parameter (run open excel cell or not )
# OpenEditData=True


In [331]:
national = ['National'] if "NATIONAL" in areas else []
retailers = regions_RET + channels_RET + market_RET
channels = regions_CHAN + channels_CHAN + market_CHAN
cust = regions_CUST + channels_CUST + market_CUST
area=national+retailers+channels+cust

In [332]:
loaded_data = {}

datasets_path = os.getcwd()+"\\""Financials Datasets Test""\\"
print(datasets_path)

datasets = os.listdir(datasets_path)
for dataset in datasets:
    file_path = os.path.join(datasets_path, dataset)
    with open(file_path, 'rb') as handle:
        globals()[dataset.split('.')[0]] = pd.read_pickle(handle)

c:\Users\Ali Salem\Desktop\App_Update\Financials\Financials Datasets Test\


### Replacement functions

# Slide1: Mix Analysis

### By Brand

In [333]:
modified_brand_P12M_dfs={}
for k in brands_client_dfs.keys():
    t=brands_client_dfs[k].copy()
    t=t[:-1]
    t['Value Sales IYA'] = t['Value Sales IYA'].astype(float).fillna(-199)
    t=t.fillna(0)
    t = t[t['Net Sales'].astype(float) >= 1000]
    total= t[(t['Top Brands'].str.contains( ' Total')) & ~(t['Top Brands'].isin(['Grand Total','All Others Total'])) & ~(t['Top Brands'].isin([i+' Total' for i in client_brands]))]
    if not t.empty and len(t['Top Brands']) != 1:
        modified_brand_P12M_dfs[k]=t

### By Market 

In [334]:
def process_RET_CHAN_POS(dataframes,regions, channels, markets):
    modified_regions = {}
    modified_channels = {}
    modified_markets = {}
    dya = {}

    for k in dataframes.keys():
        t = dataframes[k].copy()
        t = t[t["Net Sales"].astype(float)>=1000]
        t['Value Sales IYA'] =t['Value Sales IYA'].astype(float).fillna(-199) 

        level_check = set(["Region", "Channel", "Market"])
        existant_cols = list(set(t.columns)&level_check)

        decider_dic = {"Region": regions, "Channel":channels, "Market":markets}
        grand_tot = t[t[t.columns[0]] == 'Grand Total'] 
        dya[k] = grand_tot

        if len(existant_cols) == 1:
            df = t[t[t.columns[0]].isin(decider_dic[existant_cols[0]])]
            if not df.empty:
                if df.columns[0] == "Region":
                    modified_regions[k] = df
                if df.columns[0] == "Channel":
                    modified_channels[k] = df
                if df.columns[0] == "Market":
                    modified_markets[k] = df

        elif len(existant_cols) == 2:
            levels_rank = {"Region":1, "Channel":2, "Market":3}
            existant_cols = sorted(existant_cols, key=lambda x: levels_rank[x])
            t_child = t[t[existant_cols[1]].isin(decider_dic[existant_cols[1]])].drop(columns = [existant_cols[0]])
            t_parent = t[t[existant_cols[0]].isin([i + " Total" for i in decider_dic[existant_cols[0]]])].drop(columns = [existant_cols[1]])
            t_parent[existant_cols[0]] = t_parent[existant_cols[0]].str.replace(" Total", "").str.strip()
            for df in [t_child, t_parent]:
                if not df.empty:
                    if df.columns[0] == "Region":
                        modified_regions[k] = df
                    if df.columns[0] == "Channel":
                        modified_channels[k] = df
                    if df.columns[0] == "Market":
                        modified_markets[k] = df
                        
            print(k,df)
            

        else:
            levels_rank = {"Region":1, "Channel":2, "Market":3}
            existant_cols = sorted(existant_cols, key=lambda x: levels_rank[x])
            t_market = t[t[existant_cols[2]].isin(decider_dic[existant_cols[2]])].drop(columns = [existant_cols[0],existant_cols[1]])
            t_channel = t[t[existant_cols[1]].isin([i + " Total" for i in decider_dic[existant_cols[1]]])].drop(columns = [existant_cols[0],existant_cols[2]])
            t_channel[existant_cols[1]] = t_channel[existant_cols[1]].str.replace(" Total", "").str.strip()
            t_region = t[t[existant_cols[0]].isin([i + " Total" for i in decider_dic[existant_cols[0]]])].drop(columns = [existant_cols[1],existant_cols[2]])
            t_region[existant_cols[0]] = t_region[existant_cols[0]].str.replace(" Total", "").str.strip()
            if not t_region.empty:
                modified_regions[k] = t_region
            if not t_channel.empty:
                modified_channels[k] = t_channel
            if not t_market.empty:
                modified_markets[k] = t_market
            
    return modified_regions, modified_channels, modified_markets, dya
            



In [335]:
modified_retailer_regions = {}
modified_retailer_channels = {}
modified_retailer_markets = {}

modified_channels_regions = {}
modified_channels_channels = {}
modified_channels_markets = {}

modified_cust_regions = {}
modified_cust_channels = {}
modified_cust_markets = {}

dya_retailer={}
dya_channel={}
dya_cust = {}

#**********Retailer*********
if len(retailers)!=0:
    modified_retailer_regions, modified_retailer_channels, modified_retailer_markets, dya_retailer = process_RET_CHAN_POS(retailers_P12M_dfs, regions_RET, channels_RET, market_RET)
# *********Channels**********
if len(channels)!=0:
    modified_channels_regions, modified_channels_channels, modified_channels_markets, dya_channel = process_RET_CHAN_POS(channel_P12M_dfs, regions_CHAN, channels_CHAN, market_CHAN)
#********POS****************
if len(cust)!=0: 
    modified_cust_regions, modified_cust_channels, modified_cust_markets, dya_cust = process_RET_CHAN_POS(cust_P12M_dfs, regions_CUST, channels_CUST, market_CUST)


Pbg | Razors Empty DataFrame
Columns: [Channel, Gross Margin %, Gross Margin % IYA, Net Sales, Net Sales/Kg, Net Sales/Kg IYA, Value Sales IYA]
Index: []
Pbg | System Empty DataFrame
Columns: [Channel, Gross Margin %, Gross Margin % IYA, Net Sales, Net Sales/Kg, Net Sales/Kg IYA, Value Sales IYA]
Index: []
Pbg | Refills Empty DataFrame
Columns: [Channel, Gross Margin %, Gross Margin % IYA, Net Sales, Net Sales/Kg, Net Sales/Kg IYA, Value Sales IYA]
Index: []
Pbg | Disposables Empty DataFrame
Columns: [Channel, Gross Margin %, Gross Margin % IYA, Net Sales, Net Sales/Kg, Net Sales/Kg IYA, Value Sales IYA]
Index: []
Pbg | Manual Shave Men Empty DataFrame
Columns: [Channel, Gross Margin %, Gross Margin % IYA, Net Sales, Net Sales/Kg, Net Sales/Kg IYA, Value Sales IYA]
Index: []


### By Sector/Segment

In [336]:
def MixAnalysisCleaning(inputdic):
    modifieddic={}
    for k in inputdic.keys():
        t=inputdic[k].copy()
        t=t[:-1]
        t['Value Sales IYA'] =t['Value Sales IYA'].astype(float).fillna(-199)
        t=t.fillna(0)
        t=t[t["Net Sales"]>=1000]
        if not t.empty:
            modifieddic[k]=t
    return modifieddic   


In [337]:
if len(sectors)!=0:
    modified_sectors_P12M_mix_analysis_dfs=MixAnalysisCleaning(Sector_P12M_client_dfs)
if len(segments)!=0:
    modified_segment_P12M_mix_analysis_dfs=MixAnalysisCleaning(Segment_P12M_client_dfs)
if len(subsegments)!=0:
    modified_subsegment_P12M_mix_analysis_dfs=MixAnalysisCleaning(SubSegment_P12M_client_dfs)
if len(subcategories)!=0:
    modified_subcategory_P12M_mix_analysis_dfs=MixAnalysisCleaning(SubCategory_P12M_client_dfs)

# Slide2: Trade Margin

In [338]:
def TradeMarginCleaning(inputdic,scope=""):
    modifieddic={}
    for k in inputdic.keys():
        t = inputdic[k].copy()
        t[scope] = t[scope].replace('Grand Total', 'Total')
        t=t.replace(np.nan,0)
        if not t.empty:
            modifieddic[k]=t
    return modifieddic   


In [339]:
if len(sectors)!=0:
    modified_sector_Trade_Margin_dfs=TradeMarginCleaning(Sector_P12M_client_dfs,scope="Sector")
if len(segments)!=0:
    modified_segment_Trade_Margin_dfs=TradeMarginCleaning(Segment_P12M_client_dfs,scope="Segment")

if len(subsegments)!=0:
    modified_subsegment_Trade_Margin_dfs=TradeMarginCleaning(SubSegment_P12M_client_dfs,scope="SubSegment")

if len(subcategories)!=0:
    modified_subcategory_Trade_Margin_dfs=TradeMarginCleaning(SubCategory_P12M_client_dfs,scope="SubCategory")


# Sector KPI

#### Cleaning

In [340]:
def clean(data, Scope ="Sector"):
    final ={}
    for key, df in data.items():
        df.fillna(0,inplace = True)
        df = df.reset_index(drop=True)
        df =df[df['Net Sales'] > 1000]

        df = df[~df[Scope].str.contains('Grand Total', case=False)]
        if df.shape[0] !=0:
            if 'National' in key:
                new_key = key.split(' | ')[0] + ' | ' + key.split(' | ')[2] +' | ' + key.split(' | ')[1]
            else:
                new_key = key
            final[new_key] = df
    return final

In [341]:
def merge_data(data1,data2, Scope = "Sector"):
    final = {}
    for key in modified_sector_P12M.keys():
        if key in modified_sector_P3M.keys():
            final[key] = pd.merge(data1[key], data2[key], on = Scope, suffixes= ("_P12M", "_P3M"))
            df = final[key]
            df = df[~df[Scope].str.contains('Total', case=False)]
            df =df[df['Net Sales_P3M'] > 1000]
            
            df = df.sort_values('Rate of Sales_P3M', ascending=False).reset_index(drop =True)
            
            
            if df.shape[0]>0:
                final[key] = df
            
    return final

In [342]:
modified_sector_P12M = {}
modified_sector_P3M ={}
merged_data_sector ={}
if len(sectors)!=0:
    modified_sector_P12M  = clean(Sector_P12M_client_dfs, "Sector")
    modified_sector_P3M  = clean(Sector_P3M_client_dfs, "Sector")
    merged_data_sector = merge_data(modified_sector_P12M,modified_sector_P3M, "Sector")


modified_segment_P12M ={}
modified_segment_P3M ={}
merged_data_segment ={}
if len(segments)!=0:
    modified_segment_P12M  = clean(Segment_P12M_client_dfs, "Segment")
    modified_segment_P3M  = clean(Segment_P3M_client_dfs, "Segment")
    merged_data_segment = merge_data(modified_segment_P12M,modified_segment_P3M, "Segment")

modified_subsegment_P12M ={}
modified_subsegment_P3M ={}
merged_data_subsegment ={}
if len(subsegments)!=0:
    modified_subsegment_P12M  = clean(SubSegment_P12M_client_dfs, "SubSegment")
    modified_subsegment_P3M  = clean(SubSegment_P3M_client_dfs, "SubSegment")
    merged_data_subsegment = merge_data(modified_subsegment_P12M,modified_subsegment_P3M, "SubSegment")

modified_subcategory_P12M = {}
modified_subcategory_P3M ={}
merged_data_subcategory ={}
if len(subcategories)!=0:
    modified_subcategory_P12M  = clean(SubCategory_P12M_client_dfs, "SubCategory")
    modified_subcategory_P3M  = clean(SubCategory_P3M_client_dfs, "SubCategory")
    merged_data_subcategory = merge_data(modified_subcategory_P12M,modified_subcategory_P3M, "SubCategory")


# Slide 4: SKU KPIs Summary 

### By_Product

In [343]:
modified_product_P12M_dfs={}
modified_product_P3M_dfs={}
modified_product_dfs = {}

#P3M
for k in product_P3M_dfs.keys():
    t=product_P3M_dfs[k].copy()
    t=t[:-1]
    t['Top Brands']=t['Top Brands'].ffill()
    t = t[~t["Top Brands"].str.contains(' Total')]
    df = t[t['Top Brands'].isin(client_brands)]
    df = df.fillna(0)
    df = df[df['Net Sales'] > 1000]
    if not df.empty:
        modified_product_P3M_dfs[k] = df

#P12M
for k in product_P12M_dfs.keys():
    t=product_P12M_dfs[k].copy()
    t=t[:-1]
    t['Top Brands']=t['Top Brands'].ffill()
    t = t[~t["Top Brands"].str.contains(' Total')]
    df = t[t['Top Brands'].isin(client_brands)]
    df = df.fillna(0)
    df['Net Sales'] = df['Net Sales'].astype(float)
    df = df[df['Net Sales'] > 1000]
    if not df.empty:
        modified_product_P12M_dfs[k] = df


missing_keys = [k for k in modified_product_P12M_dfs.keys() if k not in modified_product_P3M_dfs]
for k in missing_keys:
    modified_product_P3M_dfs[k] = pd.DataFrame(columns=modified_product_P12M_dfs[k].columns)

for k in modified_product_P12M_dfs.keys():
    p12m_df = modified_product_P12M_dfs[k].copy()
    p12m_df = p12m_df.nlargest(15, 'Net Sales')
    p3m_df = modified_product_P3M_dfs[k].copy()
    comb = pd.merge(p12m_df,p3m_df, how = 'left',on = ["Top Brands",f'{prodORitem}'], suffixes = ("_P12M", "_P3M"))
    comb = comb.sort_values(by=f'{prodORitem} Sales Rate_P3M', ascending=False)
    # Compute Sales Rate Ix
    first_value = comb[f'{prodORitem} Sales Rate_P3M'].iloc[0]
    comb['Sales Rate Ix'] = (comb[f'{prodORitem} Sales Rate_P3M'] / first_value)
    modified_product_dfs[k] = comb.fillna(0)

### By_Brand 


In [344]:
modified_brand_product_P3M_dfs={}
modified_brand_product_P12M_dfs={}
modified_brand_product_dfs={}

###P3M
for k in modified_product_P3M_dfs.keys():
        t = modified_product_P3M_dfs[k].copy()
        for brand in client_brands:
            df = t[t['Top Brands'] == brand]
            new_key = f"{brand} | {k}"
            parts = [part.strip() for part in new_key.split('|') if f'{client_manuf[0]}' not in part]
            new_key = ' | '.join(parts)
            if not df.empty :
                modified_brand_product_P3M_dfs[new_key] = df  


###P12M    
for k in modified_product_P12M_dfs.keys():
        t = modified_product_P12M_dfs[k].copy()
        for brand in client_brands:
            df = t[t['Top Brands'] == brand]
            new_key = f"{brand} | {k}"
            parts = [part.strip() for part in new_key.split('|') if f'{client_manuf[0]}' not in part]
            new_key = ' | '.join(parts)
            if not df.empty :
                modified_brand_product_P12M_dfs[new_key] = df    


missing_keys = [k for k in modified_brand_product_P12M_dfs.keys() if k not in modified_brand_product_P3M_dfs]
for k in missing_keys:
    modified_brand_product_P3M_dfs[k] = pd.DataFrame(columns=modified_brand_product_P12M_dfs[k].columns)

for k in modified_brand_product_P12M_dfs.keys():
    p12m_df = modified_brand_product_P12M_dfs[k].copy()
    p12m_df = p12m_df.nlargest(15, 'Net Sales')
    p3m_df = modified_brand_product_P3M_dfs[k].copy()
    comb = pd.merge(p12m_df,p3m_df, how = 'left',on = ["Top Brands",f'{prodORitem}'], suffixes = ("_P12M", "_P3M"))
    
    comb = comb.sort_values(by=f'{prodORitem} Sales Rate_P3M', ascending=False)
    # Compute Sales Rate Ix
    first_value = comb[f'{prodORitem} Sales Rate_P3M'].iloc[0]
    comb['Sales Rate Ix'] = (comb[f'{prodORitem} Sales Rate_P3M'] / first_value)
    modified_brand_product_dfs[k] = comb.fillna(0)          

# Slide 5_Mix Matrix

In [345]:
def mixMatrixClean(data):
    final ={}
    for key, val in data.items():
        df=val.copy()
        if df.shape[0] !=0:
            if 'National' in key:
                new_key = ' | '.join([key.split(' | ')[0],key.split(' | ')[2],key.split(' | ')[1]])
            else:
                new_key = key

            df.fillna(0,inplace = True)
            df['Source'] = new_key.split(' | ')[2]

            final[new_key] = df.sort_values(by='Value Sales',ascending=False)
    #         final[new_key] = df.sort_values(by='Value Sales',ascending=False)
    sortOrder=final[new_key][final[new_key].columns[0]].unique()

    # print(sortOrder,final[new_key].columns[0])
    for key,val in final.items():
        val['order']=val[val.columns[0]].replace(dict(zip(sortOrder,range(len(sortOrder)))))
        final[key]=val



    return final


In [346]:
if len(sectors)!=0:
    sector_P12M_modified = mixMatrixClean(Sector_P12M_client_dfs)
if len(segments)!=0:
    segment_P12M_modified = mixMatrixClean(Segment_P12M_client_dfs)
if len(subcategories)!=0:
    subCat_P12M_modified = mixMatrixClean(SubCategory_P12M_client_dfs)
if len(subsegments)!=0:
    subSeg_P12M_modified = mixMatrixClean(SubSegment_P12M_client_dfs)
    
    
secRetailerDf,segRetailerDf,subCatRetailerDf,subSegRetailerDf,brandsRetailerDf={},{},{},{},{}
secChannelDf,segChannelDf,subCatChannelDf,subSegChannelDf,brandsChannelDf={},{},{},{},{}
secCustDf,segCustDf,subCatCustDf,subSegCustDf,brandsCustDf={},{},{},{},{}

for key,retailerList in {'region':['National'] if national else []+regions_RET,'channel':['National'] if national else []+channels_RET,'market':['National'] if national else []+market_RET}.items():
    if len(sectors)!=0:secRetailerDf[key] = {key: sector_P12M_modified[key] for key in sector_P12M_modified.keys()  if (key.split(' | ')[2] in retailerList)}
    if len(segments)!=0:segRetailerDf[key] = {key: segment_P12M_modified[key] for key in segment_P12M_modified.keys()  if (key.split(' | ')[2] in retailerList)}
    if len(subcategories)!=0:subCatRetailerDf[key] = {key: subCat_P12M_modified[key] for key in subCat_P12M_modified.keys()  if (key.split(' | ')[2] in retailerList)}
    if len(subsegments)!=0:subSegRetailerDf[key] = {key: subSeg_P12M_modified[key] for key in subSeg_P12M_modified.keys()  if (key.split(' | ')[2] in retailerList)}

for key,channelList in {'region':['National'] if national else []+regions_CHAN,'channel':['National'] if national else []+channels_CHAN,'market':['National'] if national else []+market_CHAN}.items():
    if len(sectors)!=0:secChannelDf[key] = {key: sector_P12M_modified[key] for key in sector_P12M_modified.keys()  if (key.split(' | ')[2] in channelList)}
    if len(segments)!=0:segChannelDf[key] = {key: segment_P12M_modified[key] for key in segment_P12M_modified.keys()  if (key.split(' | ')[2] in channelList)}
    if len(subcategories)!=0:subCatChannelDf[key] = {key: subCat_P12M_modified[key] for key in subCat_P12M_modified.keys()  if (key.split(' | ')[2] in channelList)}
    if len(subsegments)!=0:subSegChannelDf[key] = {key: subSeg_P12M_modified[key] for key in subSeg_P12M_modified.keys()  if (key.split(' | ')[2] in channelList)}

for key,custList in {'region':['National'] if national else []+regions_CUST,'channel':['National'] if national else []+channels_CUST,'market':['National'] if national else []+market_CUST}.items():
    if len(sectors)!=0:secCustDf[key] = {key: sector_P12M_modified[key] for key in sector_P12M_modified.keys()  if (key.split(' | ')[2] in custList)}
    if len(segments)!=0:segCustDf[key] = {key: segment_P12M_modified[key] for key in segment_P12M_modified.keys()  if (key.split(' | ')[2] in custList)}
    if len(subcategories)!=0:subCatCustDf[key] = {key: subCat_P12M_modified[key] for key in subCat_P12M_modified.keys()  if (key.split(' | ')[2] in custList)}
    if len(subsegments)!=0:subSegCustDf[key] = {key: subSeg_P12M_modified[key] for key in subSeg_P12M_modified.keys()  if (key.split(' | ')[2] in custList)}


In [347]:
secRetailerClient,segRetailerClient,subCatRetailerClient,subSegRetailerClient={},{},{},{}
secChannelClient,segChannelClient,subCatChannelClient,subSegChannelClient={},{},{},{}
secCustClient,segCustClient,subCatCustClient,subSegCustClient={},{},{},{}


def clientAgg(dic):

    clients={}
    for marketLevel in ['region','channel','market']:
        for manuf in client_manuf:
            keys = [dic[marketLevel][key] for key in dic[marketLevel].keys() if manuf in key.split(' | ')]
            if keys:
                clients[manuf+' | '+marketLevel]=pd.concat(keys).reset_index(drop = True)
    return clients

if len(sectors)!=0:secRetailerClient = clientAgg(secRetailerDf)
if len(segments)!=0:segRetailerClient = clientAgg(segRetailerDf)
if len(subcategories)!=0:subCatRetailerClient = clientAgg(subCatRetailerDf)
if len(subsegments)!=0:subSegRetailerClient = clientAgg(subSegRetailerDf)

if len(sectors)!=0:secChannelClient = clientAgg(secChannelDf)
if len(segments)!=0:segChannelClient = clientAgg(segChannelDf)
if len(subcategories)!=0:subCatChannelClient = clientAgg(subCatChannelDf)
if len(subsegments)!=0:subSegChannelClient = clientAgg(subSegChannelDf)

if len(sectors)!=0:secCustClient = clientAgg(secCustDf)
if len(segments)!=0:segCustClient = clientAgg(segCustDf)
if len(subcategories)!=0:subCatCustClient = clientAgg(subCatCustDf)
if len(subsegments)!=0:subSegCustClient = clientAgg(subSegCustDf)


In [348]:
retailerDuplication = sum([len(secRetailerClient)+len(segRetailerClient)+len(subCatRetailerClient)+len(subSegRetailerClient)])
channelDuplication = sum([len(secChannelClient)+len(segChannelClient)+len(subCatChannelClient)+len(subSegChannelClient)])
custDuplication = sum([len(secCustClient)+len(segCustClient)+len(subCatCustClient)+len(subSegCustClient)])
retailerDuplication,channelDuplication,custDuplication

(4, 0, 2)

In [349]:
def fillMissingValue(dfToFill):
    for key,val in dfToFill.items():
        colToFill = val.columns[0]
        uniqueValue = val[~val[colToFill].str.contains('Total')][colToFill].unique()
        dfLis = []
        for source in val['Source'].unique():
            df = val[val.Source==source]
            missingValue = list(set(uniqueValue) - set(df[~df[colToFill].str.contains('Total')][colToFill].unique()))
            missingValue = pd.DataFrame({colToFill: missingValue,'Source':source})
            val = pd.concat([val, missingValue]).replace(np.nan,0).reset_index(drop=True)
            
        ## Value Identifier on the avg or national value for the color schema on replacement
        if 'National' in val['Source'].unique():
            val['Net Total'] = val[(val.Source=='National')&(val[colToFill].str.contains('Total'))]['Net Sales/Kg'].iloc[0]
            val['GM Total'] = val[(val.Source=='National')&(val[colToFill].str.contains('Total'))]['Gross Margin %'].iloc[0]
        else:
            val['Net Total'] = val[(val[colToFill].str.contains('Total'))]['Net Sales/Kg'].sum()/val[(val[colToFill].str.contains('Total'))]['Net Sales/Kg'].count()
            val['GM Total'] = val[(val[colToFill].str.contains('Total'))]['Gross Margin %'].sum()/val[(val[colToFill].str.contains('Total'))]['Gross Margin %'].count()
        dfToFill[key]=val
        
    return dfToFill
                            
retailerDic = {'Sec':secRetailerClient,'Seg':segRetailerClient,'SubSeg':subSegRetailerClient,'SubCat':subCatRetailerClient}
channelDic = {'Sec':secChannelClient,'Seg':segChannelClient,'SubSeg':subSegChannelClient,'SubCat':subCatChannelClient}
custDic = {'Sec':secCustClient,'Seg':segCustClient,'SubSeg':subSegCustClient,'SubCat':subCatCustClient}

for key,val in retailerDic.items():
    retailerDic[key] = fillMissingValue(val)
for key,val in channelDic.items():
    channelDic[key] = fillMissingValue(val)
for key,val in custDic.items():
    custDic[key] = fillMissingValue(val)

# SLide6: MixMatrix ByBrand

In [350]:
def mixMatrixbybrandClean(data):
    final = {}
    for key, val in data.items():
        df = val.copy()       
        if df.shape[0] != 0:
            new_key = key
            df.fillna(0, inplace=True)
            df['Source'] = new_key.split(' | ')[1]
            # Sort the DataFrame by 'Value Sales'
            final[new_key] = df.sort_values(by='Value Sales', ascending=False)
            # Generate sort order based on sorted unique values of the first column
            sortOrder = final[new_key][final[new_key].columns[0]].unique()
            # Apply the sort order to the current DataFrame
            final[new_key]['order'] = final[new_key][final[new_key].columns[0]].replace(dict(zip(sortOrder, range(len(sortOrder)))))
 
    return final
 

In [351]:
def fillMissingValue(dfToFill):
    for key,val in dfToFill.items():
        colToFill = val.columns[0]
        uniqueValue = val[~val[colToFill].str.contains('Total')][colToFill].unique()
        dfLis = []
        for source in val['Source'].unique():
            df = val[val.Source==source]
            missingValue = list(set(uniqueValue) - set(df[~df[colToFill].str.contains('Total')][colToFill].unique()))
            missingValue = pd.DataFrame({colToFill: missingValue,'Source':source})
            val = pd.concat([val, missingValue]).replace(np.nan,0).reset_index(drop=True)
            
        ## Value Identifier on the avg or national value for the color schema on replacement
            val['Net Total'] = val[(val[colToFill].str.contains('Total'))]['Net Sales/Kg'].sum()/val[(val[colToFill].str.contains('Total'))]['Net Sales/Kg'].count()
            val['GM Total'] = val[(val[colToFill].str.contains('Total'))]['Gross Margin %'].sum()/val[(val[colToFill].str.contains('Total'))]['Gross Margin %'].count()
        dfToFill[key]=val
        
    return dfToFill
            

In [352]:
# cat,sec,seg,subseg,subcat 
secbrand,segbrand,subsegbrand,subcatbrand = {},{},{},{}
for key in brands_client_dfs.keys():
    key_split = key.split(' | ')
    if len(sectors) != 0 and len(key_split) > 2 and key_split[1] in sectors or key_split[1] in categories  :secbrand[key] = brands_client_dfs[key]
    if len(segments) != 0 and len(key_split) > 2 and key_split[1] in segments or key_split[1] in categories:segbrand[key] = brands_client_dfs[key]
    if len(subsegments) != 0 and len(key_split) > 2 and key_split[1] in subsegments or key_split[1] in categories:subsegbrand[key] = brands_client_dfs[key]
    if len(subcategories) != 0 and len(key_split) > 2 and key_split[1] in subcategories or key_split[1] in categories:subcatbrand[key] = brands_client_dfs[key]
 
if len(sectors) != 0:
        sec_dfclean=mixMatrixbybrandClean(secbrand)
        sectorbybrand=fillMissingValue(sec_dfclean)
if len(segments) != 0:
        seg_dfclean=mixMatrixbybrandClean(segbrand)
        segmentbybrand=fillMissingValue(seg_dfclean)

if len(subsegments) != 0: 
        subseg_dfclean=mixMatrixbybrandClean(subsegbrand)
        subsegmentbybrand=fillMissingValue(subseg_dfclean)

if len(subcategories) != 0:
       subcat_dfclean=mixMatrixbybrandClean(subcatbrand)
       subcategorybybrand=fillMissingValue(subcat_dfclean)


In [353]:
def concatdf(dic,area):
    final = {}
    for key_part in area:
        aligned_dfs = []

        # Filter and align DataFrames for the current area
        for key, df in dic.items():
            key_parts = key.split(" | ")  # Split key into parts based on " | "
            if key_part in key_parts:  # Check if the key includes the area (e.g., "NATIONAL")
                # Set 'Top Brands' as index for alignment, reindex to ensure all brands are present
                area_brands = pd.concat([df['Top Brands'] for key, df in dic.items() if key_part in key]).unique()
                aligned_df = df.set_index('Top Brands').reindex(area_brands).reset_index()
                aligned_df['Source'] = aligned_df['Source'].ffill()  # Track the source of the data
                aligned_df = aligned_df.fillna(0)  # Fill missing values with 0
                aligned_df.rename(columns={"Top Brands": client_manuf[0]}, inplace=True)
                aligned_dfs.append(aligned_df)

        if aligned_dfs:  # Ensure there's at least one DataFrame to concatenate
            final_df = pd.concat(aligned_dfs, ignore_index=True)
            final[key_part] = final_df  # Store the concatenated DataFrame in the final dictionary

    return final

In [354]:
if len(sectors)!=0:
    secfinaldic=concatdf(sectorbybrand,area)
if len(segments)!=0:
    segfinaldic=concatdf(segmentbybrand,area)
if len(subsegments)!=0:
    subsegfinaldic=concatdf(subsegmentbybrand,area)
if len(subcategories)!=0:
    subcatfinaldic=concatdf(subcategorybybrand,area)

# Slide7:Sector Spending Pool

### Product Spending Pool

#### Cleaning

In [355]:
def cleanProducts(data):
    final ={}
    for key, df in data.items():
        df['Top Brands'] = df['Top Brands'].replace(0, np.nan)
        df['Top Brands'] = df['Top Brands'].fillna(method = 'ffill')
        df = df[~df['Top Brands'].str.contains('Total', case=False)]
        df.fillna(0,inplace = True)
        df = df[df['Net Sales']>1000]
        df = df.sort_values(by= 'Net Sales', ascending=False)
        df =df.head(12)
        df = df.reset_index(drop=True)
        if df.shape[0] !=0:
            if 'National' in key:
                new_key = key.split(' | ')[0] + ' | ' + key.split(' | ')[2] +' | ' + key.split(' | ')[1]
            else:
                new_key = key
            final[new_key] = df
    return final

In [356]:
modified_products_p12m = cleanProducts(product_P12M_dfs)

In [357]:
modified_products_p12m_brand ={}
for key, df in modified_brand_product_P12M_dfs.items():
    df['Top Brands'] = df['Top Brands'].replace(0, np.nan)
    df['Top Brands'] = df['Top Brands'].fillna(method = 'ffill')
    df = df[~df['Top Brands'].str.contains('Total', case=False)]
    df.fillna(0,inplace = True)
    df = df[df['Net Sales']>1000]
    df = df[df['Top Brands'].isin(client_brands)]
    df = df.sort_values(by= 'Net Sales', ascending=False)
    df =df.head(12)
    df = df.reset_index(drop=True)
    if df.shape[0] !=0:
        if 'National' in key:
            new_key = key.split(' | ')[0] + ' | ' + key.split(' | ')[2] +' | ' + key.split(' | ')[1]
        else:
            new_key = key
        modified_products_p12m_brand[new_key] = df



# SKU Profitability Slide 8

### By Product

In [358]:
###P12M
modified_product_sec_seg_P12M_dfs = {}

for k in product_P12M_dfs.keys():
    t=product_P12M_dfs[k].copy()
    t=t[:-1]
    t['Value Sales IYA'] =t['Value Sales IYA'].astype(float).fillna(-199)
    t['Net Sales'] =t['Net Sales'].astype(float).fillna(0)
    t['Net Sales/Kg'] =t['Net Sales/Kg'].astype(float).fillna(0)

    t['Gross Margin %'] =t['Gross Margin %'].astype(float).fillna(0)

    t['Top Brands']=t['Top Brands'].ffill()
    total= t[(t['Top Brands'].str.contains( ' Total')) & ~(t['Top Brands'].isin(['Grand Total','All Others Total'])) & ~(t['Top Brands'].isin([i+' Total' for i in client_brands]))]
    df = t[t['Top Brands'].isin(client_brands)]
    df[f'{prodORitem} Sales Rate'] = df[f'{prodORitem} Sales Rate'].fillna(0)
    df = df[df['Net Sales'] >= 1000]
    if (not df.empty):
        modified_product_sec_seg_P12M_dfs[k] = df


# Slide9: Trade margin table vs Competition 

### By client and competitor 

In [359]:
def sort_df_excluding_total(df, Inscope=""):
    df = df.fillna(0)
    total_row = df[df[f'{ManufOrTopC}'] == 'Total']
    df = df[df[f'{ManufOrTopC}'] != 'Total']
    df = df.sort_values(by=[Inscope], ascending=[True]).reset_index(drop=True)
    df = pd.concat([df, total_row]).reset_index(drop=True)

    return df

def split_client_copetitor(dictionary,client_dictionary,competitor_dictionary,Inscope=""):
    for k in dictionary.keys():
        t = dictionary[k].copy()
        t=t[:-1]
        t[Inscope]=t[Inscope].fillna(t[f'{ManufOrTopC}'])
        t['Net Sales'] = t['Net Sales'].astype(float)
        total_entries = t[(t[f'{ManufOrTopC}'].str.contains(' Total')) & ~(t[f'{ManufOrTopC}'].isin(['Grand Total',"All Others Total"])) & ~(t[f'{ManufOrTopC}'].isin([i+' Total' for i in client_manuf]))]
        total_entries['Value Share'] = total_entries['Value Share'].astype(float)
        total_entries = total_entries.nlargest(1,columns="Value Share")    
        comp_lis = list(total_entries[f'{ManufOrTopC}'].str.replace(" Total",'').str.strip())
        print(comp_lis)
        dc = (t[t[f'{ManufOrTopC}'].isin(client_manuf) | t[f'{ManufOrTopC}'].isin([i + " Total" for i in client_manuf])]).replace([i + " Total" for i in client_manuf],"Total")
        df = (t[t[f'{ManufOrTopC}'].isin(comp_lis) | t[f'{ManufOrTopC}'].isin([i + " Total" for i in comp_lis])]).replace([i + " Total" for i in comp_lis],"Total")
        
        dc = dc[dc['Net Sales'] > 1000]
        unique_scope = dc[Inscope].unique().tolist()

        df = df[df[Inscope].isin(unique_scope)]

        for subcat in unique_scope:
            print(subcat)
            if (subcat not in df[Inscope].values) & (subcat!=""):
                df = pd.concat([df, pd.DataFrame({f'{ManufOrTopC}': [df[f'{ManufOrTopC}'].unique()[0]], Inscope: [subcat]})])


        dc = sort_df_excluding_total(dc, Inscope)
        df = sort_df_excluding_total(df, Inscope)

        if not dc.empty:
            client_dictionary[k]=dc
        if not df.empty:
            competitor_dictionary[k]=df

In [360]:
####Sector_Cleaning 
client_manuf_sector_dfs_new = {}
top_competitor_manuf_sector_dfs_new = {}
if len(sectors)!=0:
    split_client_copetitor(manuf_Sector_dfs,client_manuf_sector_dfs_new,top_competitor_manuf_sector_dfs_new,Inscope="Sector")

####Segment_Cleaning
client_manuf_segment_dfs_new = {}
top_competitor_manuf_segment_dfs_new = {}
if len(segments)!=0:
    split_client_copetitor(manuf_Segment_dfs,client_manuf_segment_dfs_new,top_competitor_manuf_segment_dfs_new,Inscope="Segment")

####SubSegment_Cleaning
client_manuf_subsegment_dfs_new = {}
top_competitor_manuf_subsegment_dfs_new = {}
if len(subsegments)!=0:
    split_client_copetitor(manuf_SubSegment_dfs,client_manuf_subsegment_dfs_new,top_competitor_manuf_subsegment_dfs_new,Inscope="SubSegment")

####SubCategory_Cleaning
client_manuf_subcategory_dfs_new = {}
top_competitor_manuf_subcategory_dfs_new = {}
if len(subcategories)!=0:
    split_client_copetitor(manuf_SubCategory_dfs,client_manuf_subcategory_dfs_new,top_competitor_manuf_subcategory_dfs_new,Inscope="SubCategory")

['Procter & Gamble']
['Procter & Gamble']
['Procter & Gamble']
['Procter & Gamble']
Disposables
System
Total
[]
[]
[]
[]
[]
['Procter & Gamble']
Disposables
System
Total
['Procter & Gamble']
Disposables
System
Total
['Procter & Gamble']
Disposables
System
Total
['Procter & Gamble']
['Procter & Gamble']
['Procter & Gamble']
['Procter & Gamble']
Disposables
Razors
Refills
Total
[]
[]
[]
[]
[]
['Procter & Gamble']
Disposables
Razors
Refills
Total
['Procter & Gamble']
Disposables
Razors
Refills
Total
['Procter & Gamble']
Disposables
Razors
Refills
Total


# duplication part 

In [361]:
index = [
    *[0]*((1 if len(client_brands)>1 and len(modified_brand_P12M_dfs)>0 else 0)+(1 if len(modified_retailer_regions)>0 else 0)+(1 if len(modified_retailer_channels)>0 else 0)+(1 if len(modified_retailer_markets)>0 else 0)
          +(1 if len(modified_channels_regions)>0 else 0)+(1 if len(modified_channels_channels)>0 else 0)+(1 if len(modified_channels_markets)>0 else 0)
          +(1 if len(modified_cust_regions)>0 else 0)+(1 if len(modified_cust_channels)>0 else 0)+(1 if len(modified_cust_markets)>0 else 0)
          +(1 if sectors else 0)+(1 if segments else 0)+(1 if subsegments else 0)+(1 if subcategories else 0)+(1 if len(client_brands)>1 else 0)),
          
    *[1]*((1 if sectors else 0)+(1 if segments else 0)+(1 if subsegments else 0)+(1 if subcategories else 0)),
#     *[2]*((1 if sectors else 0)+(1 if segments else 0)+(1 if subsegments else 0)+(1 if subcategories else 0)),
    *[3]*((1 if len(client_brands)>1 else 0)+(1 if len(client_brands)>1 else 0)),
    *[4]*((1 if retailerDuplication !=0 else 0)+(1 if channelDuplication !=0 else 0 )+(1 if custDuplication !=0 else 0 )),
    *[5]*((1 if sectors else 0)+(1 if segments else 0)+(1 if subsegments else 0)+(1 if subcategories else 0)),
    *[6]*((1 if sectors else 0)+(1 if segments else 0)+(1 if subsegments else 0)+(1 if subcategories else 0)),
    *[7]*(1 if len(client_brands)>1 else 0),
    *[8]*( 1 if len(client_brands)>1 else 0),
    *[9]*((1 if sectors else 0)+(1 if segments else 0)+(1 if subsegments else 0)+(1 if subcategories else 0))
          ]


# slide 0
duplication_1=[(len(modified_brand_P12M_dfs.keys()) if len(client_brands)>1 else 0)]
duplication_2 = [len(modified_retailer_regions.keys()) if len(regions_RET)>0 else 0, len(modified_retailer_channels) if len(channels_RET)>0 else 0, len(modified_retailer_markets) if len(market_RET)>0 else 0,
                 len(modified_channels_regions.keys()) if len(regions_CHAN)>0 else 0, len(modified_channels_channels) if len(channels_CHAN)>0 else 0, len(modified_channels_markets) if len(market_CHAN) >0 else 0,
                 len(modified_cust_regions.keys()) if len(regions_CUST)>0 else 0, (len(modified_cust_channels) if len(channels_CUST)>0 else 0), (len(modified_cust_markets) if len(market_CUST)>0 else 0)] 

duplication_3 = [(len(modified_sectors_P12M_mix_analysis_dfs.keys()) if len(sectors)>0 else 0), (len(modified_segment_P12M_mix_analysis_dfs.keys()) if len(segments)>0 else 0), (len(modified_subsegment_P12M_mix_analysis_dfs) if len(subsegments)>0 else 0), (len(modified_subcategory_P12M_mix_analysis_dfs) if len(subcategories)>0 else 0)]

duplication_4 = [len(modified_product_sec_seg_P12M_dfs.keys())]

# Slide 1
duplication_5 = [(len(modified_sector_Trade_Margin_dfs.keys()) if len(sectors)!=0 else 0), (len(modified_segment_Trade_Margin_dfs.keys()) if len(segments)!=0 else 0), (len(modified_subsegment_Trade_Margin_dfs) if len(subsegments)!=0 else 0), (len(modified_subcategory_Trade_Margin_dfs) if len(subcategories)!=0 else 0)]

#Slide 2
# duplication_6 = [(len(merged_data_sector.keys()) if len(sectors)!=0 else 0), (len(merged_data_segment.keys()) if len(segments)!=0 else 0), (len(merged_data_subsegment) if len(subsegments)!=0 else 0), (len(merged_data_subcategory) if len(subcategories)!=0 else 0)]

# Slide 3
duplication_7 = [len(modified_product_dfs.keys()),len(modified_brand_product_dfs.keys())]
#slide 4
duplication_8 = [retailerDuplication,channelDuplication,custDuplication]

# Slide 5
duplication_9=[(len(secfinaldic.keys())if len(sectors)!=0 else 0),(len(segfinaldic.keys())if len(segments)!=0 else 0),(len(subsegfinaldic.keys())if len(subsegments)!=0 else 0),(len(subcatfinaldic.keys())if len(subcategories)!=0 else 0)]

# Slide 5
duplication_10 = [(len(modified_sector_P12M.keys()) if len(sectors)!=0 else 0), (len(modified_segment_P12M.keys()) if len(segments)!=0 else 0), (len(modified_subsegment_P12M) if len(subsegments)!=0 else 0), (len(modified_subcategory_P12M) if len(subcategories)!=0 else 0)]

# Slide 6
duplication_11 = [len(modified_products_p12m)+ len(modified_products_p12m_brand)]

# Slide 7 
duplication_12 = [len(modified_product_P12M_dfs) + len(modified_brand_product_P12M_dfs)]

# Slide 8
duplication_13 = [(len(client_manuf_sector_dfs_new.keys()) if len(sectors)!=0 else 0), (len(client_manuf_segment_dfs_new.keys()) if len(segments)!=0 else 0), (len(client_manuf_subsegment_dfs_new) if len(subsegments)!=0 else 0), (len(client_manuf_subcategory_dfs_new) if len(subcategories)!=0 else 0)]


duplication = duplication_1 + duplication_2 + duplication_3 + duplication_4 + duplication_5 + duplication_7 + duplication_8+duplication_9 + duplication_10+ duplication_11+duplication_12+duplication_13 #+ duplication_6
print(duplication_2)
duplication = [item for item in duplication if item !=0]

section_1 = (["Mix Analysis by brand"] if len(client_brands)>1 and  len(modified_brand_P12M_dfs)>0 else [])
section_2 = (["Mix Analysis by Retailer for Region"] if len(modified_retailer_regions)!=0 else [])+ (["Mix Analysis by Retailer for Channel"] if len(modified_retailer_channels)!=0 else [])+ (["Mix Analysis by Retailer for Market"] if len(modified_retailer_markets)!=0 else [])+(["Mix Analysis by Channel for Region"] if len(modified_channels_regions)!=0 else [])+ (["Mix Analysis by Channel for Channel"] if len(modified_channels_channels)!=0 else [])+ (["Mix Analysis by Channel for Market"] if len(modified_channels_markets)!=0 else [])+([f"Mix Analysis by {customareas} for Region"] if len(modified_cust_regions)!=0 else [])+ ([f"Mix Analysis by {customareas} for Channel"] if len(modified_cust_channels)!=0 else [])+ ([f"Mix Analysis by {customareas} for Market"] if len(modified_cust_markets)!=0 else [])
section_3 = (["Mix Analysis by Sector"] if len(sectors)!=0 else [])+ (["Mix Analysis by Segment"] if len(segments)!=0 else [])+ (["Mix Analysis by SubSegment"] if len(subsegments)!=0 else [])+ (["Mix Analysis by SubCategory"] if len(subcategories)!=0 else [])
section_4 = ["Mix Analysis by"+ f'{prodORitem}' if len(client_brands)>1 else []]

section_5 = (["Trade Margin Analysis by Sector"] if len(sectors)!=0 else [])+ (["Trade Margin Analysis by Segment"] if len(segments)!=0 else [])+ (["Trade Margin Analysis by SubSegment"] if len(subsegments)!=0 else [])+(["Trade Margin Analysis by SubCategory"] if len(subcategories)!=0 else [])

# section_6 = (["Sector KPI"] if len(sectors)!=0 else [])+ (["Segment KPI"] if len(segments)!=0 else [])+ (["SubSegment KPI"] if len(subsegments)!=0 else [])+ (["SubCategory KPI"] if len(subcategories)!=0 else [])

section_7 = ("SKU KPIs Summary By Manufacture" if len(client_brands)>1 else [],"SKU KPIs Summary By Brand" if len(client_brands)>1 else [])

section_8 = ['Mix Matrix By Retailer'if retailerDuplication != 0 else[] ,'Mix Matrix By Channel'if channelDuplication != 0 else[],'Mix Matrix By Custom Region'if custDuplication != 0 else[]]#,*section_names_slide4]#*section_names_slide3,*section_names_slide4]

section_9=["Mix Matrix By Brands by Sector" if len(sectors)!=0 else [],"Mix Matrix By Brands by Segment" if len(segments)!=0 else [],"Mix Matrix By Brands by SubSegment"if len(subsegments)!=0 else [],"Mix Matrix By Brands by SubCategory"if len(subcategories)!=0 else []]

section_10 = (["Sector Spending Pool"] if len(sectors)!=0 else [])+ (["Segment Spending Pool"] if len(segments)!=0 else [])+ (["SubSegment Spending Pool"] if len(subsegments)!=0 else [])+ (["SubCategory Spending Pool"] if len(subcategories)!=0 else [])

section_11 = (["Product Spending Pool"] if len(client_brands)>1 else [])

section_12 = [("SKU Profitability" if len(client_brands)>1 else [])]

section_13 = (["Trade Margin Table By Sector"] if len(sectors)!=0 else [])+ (["Trade Margin Table By Segment"] if len(segments)!=0 else [])+ (["Trade Margin Table By SubSegment"] if len(subsegments)!=0 else [])+ (["Trade Margin Table By SubCategory"] if len(subcategories)!=0 else [])

section_names = [*section_1 , *section_2 , *section_3 , *section_4 , *section_5  , *section_7 , *section_8, *section_9 , *section_10 , *section_11,*section_12,*section_13]#, *section_6

section_names = [item for item in section_names if item !=[]]
 

path = os.getcwd() + "\Financials base.pptx"
new_pre = os.getcwd() + '\Financials Slide Duplicate.pptx'

[0, 0, 5, 0, 0, 0, 0, 0, 0]


In [362]:
print(index)
print(duplication)
print(section_names)
print(len(index))
print(len(duplication))
print(len(section_names))
print(sum(duplication))


[0, 0, 0, 0, 1, 1, 3, 3, 4, 4, 5, 5, 6, 6, 7, 8, 9, 9]
[5, 4, 4, 20, 12, 12, 20, 20, 4, 2, 9, 9, 4, 4, 40, 40, 4, 4]
['Mix Analysis by Retailer for Market', 'Mix Analysis by Sector', 'Mix Analysis by Segment', 'Mix Analysis bySKU', 'Trade Margin Analysis by Sector', 'Trade Margin Analysis by Segment', 'SKU KPIs Summary By Manufacture', 'SKU KPIs Summary By Brand', 'Mix Matrix By Retailer', 'Mix Matrix By Custom Region', 'Mix Matrix By Brands by Sector', 'Mix Matrix By Brands by Segment', 'Sector Spending Pool', 'Segment Spending Pool', 'Product Spending Pool', 'SKU Profitability', 'Trade Margin Table By Sector', 'Trade Margin Table By Segment']
18
18
18
217


In [363]:
if len(slides_name) >0:
    valid_sections = [(i, s) for i, s in enumerate(section_names) if s is not None]
    indices = [i for i, s in valid_sections if any(sub.lower() == s.lower() for sub in slides_name)]
    filtered_section_names = [section_names[i] for i in indices]
    filtered_duplication = [duplication[i] for i in indices]
    filtered_index = [index[i] for i in indices]
    slideDuplication(filtered_index,filtered_duplication,filtered_section_names,path,new_pre)
else:

    slideDuplication(index,duplication,section_names,path,new_pre)

In [364]:
# slideDuplication(index,duplication,section_names,path,new_pre)

In [365]:
# def calculate_position(end):
#     return sum(duplication[i] * (1 if isinstance(index[i], int) else len(index[i])) for i in range(end))

In [366]:
def calculate_position(end, use_filtered=False):
    """
    Calculate position based on duplication values.
    
    Args:
        end: The end index to calculate position up to
        use_filtered: Whether to use filtered lists (when slides_name is specified)
    """
    if use_filtered:
        try:
            # Use filtered lists when slides are specifically requested
            duplication_list = filtered_duplication
            index_list = filtered_index
        except NameError:
            # Fall back to original lists if filtered ones don't exist
            duplication_list = duplication
            index_list = index
    else:
        # Use original lists
        duplication_list = duplication
        index_list = index
    
    return sum(duplication_list[i] * (1 if isinstance(index_list[i], int) else len(index_list[i])) 
               for i in range(end))

In [367]:
prs = Presentation(new_pre)

In [368]:
p=0
if len(client_brands)>1 and len(modified_brand_P12M_dfs) >0:
    try:
        if 0 in filtered_index and 'Mix Analysis by brand' in filtered_section_names: 
            dup_list = filtered_duplication
            run_slide = True
            use_filtered = True
        else:
            run_slide = False
    except NameError:
        dup_list = duplication
        run_slide = True 

    if run_slide:
        Mixanalysis(prs,modified_brand_P12M_dfs,dup_list[p],Inscop='Brand',position = calculate_position(p,use_filtered),label_col='Top Brands')
        p+=1

if len(regions_RET)!=0 and len(modified_retailer_regions) !=0:
    try:
        if 0 in filtered_index and 'Mix Analysis by Retailer for Channel' in filtered_section_names: 
            dup_list = filtered_duplication
            run_slide = True
            use_filtered = True
        else:
            run_slide = False
    except NameError:
        dup_list = duplication
        run_slide = True 

    if run_slide:
        print(p, dup_list[p], calculate_position(p))
        Mixanalysis(prs,modified_retailer_regions,dup_list[p],Inscop='Retailer',position = calculate_position(p,use_filtered),label_col='Region')
        p+=1

if len(channels_RET)!=0:
    try:
        if 0 in filtered_index and 'Mix Analysis by Retailer for Region' in filtered_section_names: 
            dup_list = filtered_duplication
            run_slide = True
            use_filtered = True
        else:
            run_slide = False
    except NameError:
        dup_list = duplication
        run_slide = True 

    if run_slide:
        Mixanalysis(prs,modified_retailer_channels,dup_list[p],Inscop='Retailer',position = calculate_position(p,use_filtered),label_col='Channel')
        p+=1
        
if len(market_RET)!=0 and len(modified_retailer_markets)>0:
    try:
        if 0 in filtered_index and 'Mix Analysis by Retailer for Market' in filtered_section_names: 
            dup_list = filtered_duplication
            run_slide = True
            use_filtered = True
        else:
            run_slide = False
    except NameError:
        dup_list = duplication
        run_slide = True 

    if run_slide:
        Mixanalysis(prs,modified_retailer_markets,dup_list[p],Inscop='Retailer',position = calculate_position(p,use_filtered),label_col='Market')
        p+=1

if len(regions_CHAN)!=0:
    try:
        if 0 in filtered_index and 'Mix Analysis by Channel for Region' in filtered_section_names: 
            dup_list = filtered_duplication
            run_slide = True
            use_filtered = True
        else:
            run_slide = False
    except NameError:
        dup_list = duplication
        run_slide = True 

    if run_slide:
        Mixanalysis(prs,modified_channels_regions,dup_list[p],Inscop='Channel',position = calculate_position(p,use_filtered),label_col='Region')
        p+=1

if len(channels_CHAN)!=0:
    try:
        if 0 in filtered_index and 'Mix Analysis by Channel for Channel' in filtered_section_names: 
            dup_list = filtered_duplication
            run_slide = True
            use_filtered = True
        else:
            run_slide = False
    except NameError:
        dup_list = duplication
        run_slide = True 

    if run_slide:
        Mixanalysis(prs,modified_channels_channels,dup_list[p],Inscop='Channel',position = calculate_position(p,use_filtered),label_col='Channel')
        p+=1

if len(market_CHAN)!=0:
    try:
        if 0 in filtered_index and 'Mix Analysis by Channel for Market' in filtered_section_names: 
            dup_list = filtered_duplication
            run_slide = True
            use_filtered = True
        else:
            run_slide = False
    except NameError:
        dup_list = duplication
        run_slide = True 

    if run_slide:
        Mixanalysis(prs,modified_channels_markets,dup_list[p],Inscop='Channel',position = calculate_position(p,use_filtered),label_col='Market')
        p+=1


if len(regions_CUST)!=0:
    try:
        if 0 in filtered_index and 'Mix Analysis by {customareas} for Region' in filtered_section_names: 
            dup_list = filtered_duplication
            run_slide = True
            use_filtered = True
        else:
            run_slide = False
    except NameError:
        dup_list = duplication
        run_slide = True 

    if run_slide:
        Mixanalysis(prs,modified_cust_regions,dup_list[p],Inscop=f"{customareas}",position = calculate_position(p,use_filtered),label_col='Region')
        p+=1

if len(channels_CUST)!=0 and len(modified_cust_channels)!=0:
    try:
        if 0 in filtered_index and 'Mix Analysis by {customareas} for Channel' in filtered_section_names: 
            dup_list = filtered_duplication
            run_slide = True
            use_filtered = True
        else:
            run_slide = False
    except NameError:
        dup_list = duplication
        run_slide = True 

    if run_slide:
        Mixanalysis(prs,modified_cust_channels,dup_list[p],Inscop=f"{customareas}",position = calculate_position(p,use_filtered),label_col='Channel')
        p+=1

if len(market_CUST)!=0 and len(modified_cust_markets)>0:
    try:
        if 0 in filtered_index and 'Mix Analysis by {customareas} for Market' in filtered_section_names: 
            dup_list = filtered_duplication
            run_slide = True
            use_filtered = True
        else:
            run_slide = False
    except NameError:
        dup_list = duplication
        run_slide = True 

    if run_slide:
        Mixanalysis(prs,modified_cust_markets,dup_list[p],Inscop=f"{customareas}",position = calculate_position(p,use_filtered),label_col='Market')
        p+=1

if len(sectors)!=0:
    try:
        if 0 in filtered_index and 'Mix Analysis by Sector' in filtered_section_names: 
            dup_list = filtered_duplication
            run_slide = True
            use_filtered = True
        else:
            run_slide = False
    except NameError:
        dup_list = duplication
        run_slide = True 
    if run_slide:
        Mixanalysis(prs,modified_sectors_P12M_mix_analysis_dfs,dup_list[p],Inscop='Sector',position = calculate_position(p,use_filtered),label_col='Sector')
        p+=1
if len(segments)!=0:    
    try:
        if 0 in filtered_index and 'Mix Analysis by Segment' in filtered_section_names: 
            dup_list = filtered_duplication
            run_slide = True
            use_filtered = True
        else:
            run_slide = False
    except NameError:
        dup_list = duplication
        run_slide = True 
    if run_slide:     
        Mixanalysis(prs,modified_segment_P12M_mix_analysis_dfs,dup_list[p],Inscop="Segment",position = calculate_position(p,use_filtered),label_col="Segment")
        p+=1
if len(subsegments)!=0:
    try:
        if 0 in filtered_index and 'Mix Analysis by Subsegment' in filtered_section_names: 
            dup_list = filtered_duplication
            run_slide = True
            use_filtered = True
        else:
            run_slide = False
    except NameError:
        dup_list = duplication
        run_slide = True 
    if run_slide:
        Mixanalysis(prs,modified_subsegment_P12M_mix_analysis_dfs,dup_list[p],Inscop="SubSegment",position = calculate_position(p,use_filtered),label_col="SubSegment")
        p+=1
if len(subcategories)!=0:
    try:
        if 0 in filtered_index and 'Mix Analysis by Subcategory' in filtered_section_names: 
            dup_list = filtered_duplication
            run_slide = True
            use_filtered = True
        else:
            run_slide = False
    except NameError:
        dup_list = duplication
        run_slide = True 
    if run_slide:
        Mixanalysis(prs,modified_subcategory_P12M_mix_analysis_dfs,dup_list[p],Inscop="SubCategory",position = calculate_position(p,use_filtered),label_col="SubCategory")
        p+=1
if len(client_brands)>1:
    try:
        if 0 in filtered_index and "Mix Analysis by"+ f'{prodORitem}' in filtered_section_names: 
            dup_list = filtered_duplication
            run_slide = True
            use_filtered = True
        else:
            run_slide = False
    except NameError:
        dup_list = duplication
        run_slide = True 
    if run_slide:
        Mixanalysis(prs,modified_product_sec_seg_P12M_dfs,dup_list[p],Inscop=f'{prodORitem}',position = calculate_position(p,use_filtered),label_col=f'{prodORitem}')
        p+=1



In [303]:
p

0

In [369]:
if len(sectors)!=0:
    try:
        if 1 in filtered_index and 'Trade Margin Analysis by Sector' in filtered_section_names: 
            dup_list = filtered_duplication
            run_slide = True
            use_filtered = True
        else:
            run_slide = False
    except NameError:
        dup_list = duplication
        run_slide = True 
    if run_slide:
        TradeMargin(prs,modified_sector_Trade_Margin_dfs,dup_list[p],position = calculate_position(p,use_filtered),InScope='Sector')
        p+=1
if len(segments)!=0:
    try:
        if 1 in filtered_index and 'Trade Margin Analysis by Segment' in filtered_section_names: 
            dup_list = filtered_duplication
            run_slide = True
            use_filtered = True
        else:
            run_slide = False
    except NameError:
        dup_list = duplication
        run_slide = True 
    if run_slide:
        TradeMargin(prs,modified_segment_Trade_Margin_dfs,dup_list[p],position = calculate_position(p,use_filtered),InScope="Segment")
        p+=1
if len(subsegments)!=0:
    try:
        if 1 in filtered_index and 'Trade Margin Analysis by Subsegment' in filtered_section_names: 
            dup_list = filtered_duplication
            run_slide = True
            use_filtered = True
        else:
            run_slide = False
    except NameError:
        dup_list = duplication
        run_slide = True 
    if run_slide:
        TradeMargin(prs,modified_subsegment_Trade_Margin_dfs,dup_list[p],position = calculate_position(p,use_filtered),InScope="SubSegment")
        p+=1
if len(subcategories)!=0:
    try:
        if 1 in filtered_index and 'Trade Margin Analysis by Subcategory' in filtered_section_names: 
            dup_list = filtered_duplication
            run_slide = True
            use_filtered = True
        else:
            run_slide = False
    except NameError:
        dup_list = duplication
        run_slide = True 
    if run_slide:
        TradeMargin(prs,modified_subcategory_Trade_Margin_dfs,dup_list[p],position = calculate_position(p,use_filtered),InScope="SubCategory")
        p+=1        

In [305]:
# if len(sectors)!=0:
#     sectorKPI(prs, merged_data_sector, duplication[p], position=calculate_position(p),Scope= "Sector")
#     p+=1
# if len(segments)!=0:
#     sectorKPI(prs, merged_data_segment, duplication[p], position=calculate_position(p),Scope= "Segment")
#     p+=1

# if len(subsegments)!=0:
#     sectorKPI(prs, merged_data_subsegment, duplication[p], position=calculate_position(p),Scope= "SubSegment")
#     p+=1

# if len(subcategories)!=0:
#     sectorKPI(prs, merged_data_subcategory, duplication[p], position=calculate_position(p),Scope= "SubCategory") 
#     p+=1

In [370]:
p

0

In [371]:
if len(client_brands)>1:
    try:
        if 3 in filtered_index and 'SKU KPIs Summary By Manufacture' in filtered_section_names: 
            dup_list = filtered_duplication
            run_slide = True
            use_filtered = True
        else:
            run_slide = False
    except NameError:
        dup_list = duplication
        run_slide = True 
    if run_slide:
        SkuKpis(prs,modified_product_dfs,dup_list[p],position = calculate_position(p,use_filtered))
        p+=1

    try:
        if 3 in filtered_index and 'SKU KPIs Summary By Brand' in filtered_section_names: 
            dup_list = filtered_duplication
            run_slide = True
            use_filtered = True
        else:
            run_slide = False
    except NameError:
        dup_list = duplication
        run_slide = True 
    if run_slide:
        SkuKpis(prs,modified_brand_product_dfs,dup_list[p],position = calculate_position(p,use_filtered))
        p+=1

In [372]:
p

0

In [ ]:
# mixMatrix(prs,secretailerTotal,position = 0)
pos = calculate_position(p)
if retailerDuplication!=0:
    try:
        if 4 in filtered_index and 'Mix Matrix By Retailer' in filtered_section_names: 
            dup_list = filtered_duplication
            run_slide = True
            use_filtered = True
        else:
            run_slide = False
    except NameError:
        dup_list = duplication
        run_slide = True 
    if run_slide:
        for key,val in retailerDic.items():
            if val:
                pos = calculate_position(p,use_filtered)
                mixMatrix(prs,val,position = pos)
                pos +=len(retailerDic[key].keys())

        p+=1        
if channelDuplication!=0:
    try:
        if 4 in filtered_index and 'Mix Matrix By Channel' in filtered_section_names: 
            dup_list = filtered_duplication
            run_slide = True
            use_filtered = True
        else:
            run_slide = False
    except NameError:
        dup_list = duplication
        run_slide = True 
    if run_slide:
        for key,val in channelDic.items():
            if val:
                pos = calculate_position(p,use_filtered)
                mixMatrix(prs,val,position = pos)
                pos +=len(channelDic[key].keys())
        p+=1       
if custDuplication!=0:
    try:
        if 4 in filtered_index and 'Mix Matrix By Custom Region' in filtered_section_names: 
            dup_list = filtered_duplication
            run_slide = True
            use_filtered = True
        else:
            run_slide = False
    except NameError:
        dup_list = duplication
        run_slide = True 
    if run_slide:
        for key,val in custDic.items():
            if val:
                pos = calculate_position(p,use_filtered)
                mixMatrix(prs,val,position = pos)
                pos +=len(custDic[key].keys())
        p+=1        


In [375]:
p

0

In [376]:
if len(sectors)!=0:
    try:
        if 5 in filtered_index and 'Mix Matrix by Brands by Sector' in filtered_section_names: 
            dup_list = filtered_duplication
            run_slide = True
            use_filtered = True
        else:
            run_slide = False
    except NameError:
        dup_list = duplication
        run_slide = True 
    if run_slide:
        mixMatrix(prs,secfinaldic,position = calculate_position(p,use_filtered),slideby="bybrand")
        p+=1
if len(segments)!=0:
    try:
        if 5 in filtered_index and 'Mix Matrix by Brands by Segment' in filtered_section_names: 
            dup_list = filtered_duplication
            run_slide = True
            use_filtered = True
        else:
            run_slide = False
    except NameError:
        dup_list = duplication
        run_slide = True 
    if run_slide:
        mixMatrix(prs,segfinaldic,position = calculate_position(p,use_filtered),slideby="bybrand")
        p+=1
if len(subsegments)!=0:
    try:
        if 5 in filtered_index and 'Mix Matrix by Brands by Subsegment' in filtered_section_names: 
            dup_list = filtered_duplication
            run_slide = True
            use_filtered = True
        else:
            run_slide = False
    except NameError:
        dup_list = duplication
        run_slide = True 
    if run_slide:
        mixMatrix(prs,subsegfinaldic,position = calculate_position(p,use_filtered),slideby="bybrand")
        p+=1
if len(subcategories)!=0:
    try:
        if 5 in filtered_index and 'Mix Matrix by Brands by Subcategory' in filtered_section_names: 
            dup_list = filtered_duplication
            run_slide = True
            use_filtered = True
        else:
            run_slide = False
    except NameError:
        dup_list = duplication
        run_slide = True 
    if run_slide:
        mixMatrix(prs,subcatfinaldic,position = calculate_position(p,use_filtered),slideby="bybrand")
        p+=1

In [312]:
p

0

In [377]:
if len(sectors)!=0:
    try:
        if 6 in filtered_index and 'Sector Spending Pool' in filtered_section_names: 
            dup_list = filtered_duplication
            run_slide = True
            use_filtered = True
        else:
            run_slide = False
    except NameError:
        dup_list = duplication
        run_slide = True 
    if run_slide:
        sectorSpendingPool(prs, modified_sector_P12M,dup_list[p], position =calculate_position(p,use_filtered),Scope= "Sector")
        p+=1

if len(segments)!=0:
    try:
        if 6 in filtered_index and 'Segment Spending Pool' in filtered_section_names: 
            dup_list = filtered_duplication
            run_slide = True
            use_filtered = True
        else:
            run_slide = False
    except NameError:
        dup_list = duplication
        run_slide = True 
    if run_slide:
        sectorSpendingPool(prs, modified_segment_P12M,dup_list[p], position =calculate_position(p,use_filtered),Scope="Segment")
        p+=1

if len(subsegments)!=0:
    try:
        if 6 in filtered_index and 'Subsegment Spending Pool' in filtered_section_names: 
            dup_list = filtered_duplication
            run_slide = True
            use_filtered = True
        else:
            run_slide = False
    except NameError:
        dup_list = duplication
        run_slide = True 
    if run_slide:
        sectorSpendingPool(prs, modified_subsegment_P12M,dup_list[p], position =calculate_position(p,use_filtered),Scope="SubSegment")
        p+=1

if len(subcategories)!=0:
    try:
        if 6 in filtered_index and 'Subcategory Spending Pool' in filtered_section_names: 
            dup_list = filtered_duplication
            run_slide = True
            use_filtered = True
        else:
            run_slide = False
    except NameError:
        dup_list = duplication
        run_slide = True 
    if run_slide:
        sectorSpendingPool(prs, modified_subcategory_P12M,dup_list[p], position =calculate_position(p,use_filtered),Scope= "SubCategory")    
        p+=1

In [314]:
p

0

In [378]:
if len(client_brands)>1:
    combinedmanufbranddfpool=modified_products_p12m
    combinedmanufbranddfpool.update(modified_products_p12m_brand)
    try:
        if 7 in filtered_index: 
            dup_list = filtered_duplication
            run_slide = True
            use_filtered = True
        else:
            run_slide = False
    except NameError:
        dup_list = duplication
        run_slide = True 
    if run_slide:
        productSpendingPool(prs,combinedmanufbranddfpool, dup_list[p], position= calculate_position(p,use_filtered))
        p+=1

In [316]:
p

0

In [379]:
if len(client_brands)>1:
    combinedmanufbranddf=modified_product_P12M_dfs
    combinedmanufbranddf.update(modified_brand_product_P12M_dfs)
    try:
        if 8 in filtered_index: 
            dup_list = filtered_duplication
            run_slide = True
            use_filtered = True
        else:
            run_slide = False
    except NameError:
        dup_list = duplication
        run_slide = True 
    if run_slide:
        SKUProfitability(prs,combinedmanufbranddf,dup_list[p],position = calculate_position(p,use_filtered))
        p+=1

In [381]:
if len(sectors)!=0:
    try:
        if 9 in filtered_index and 'Trade Margin Table By Sector' in filtered_section_names: 
            dup_list = filtered_duplication
            run_slide = True
            use_filtered = True
        else:
            run_slide = False
    except NameError:
        dup_list = duplication
        run_slide = True 
    if run_slide:
        TradeMarginTable(prs,client_manuf_sector_dfs_new,top_competitor_manuf_sector_dfs_new,dup_list[p],position = calculate_position(p,use_filtered),Inscope="Sector")
        p+=1
if len(segments)!=0: 
    try:
        if 9 in filtered_index and 'Trade Margin Table By Segment' in filtered_section_names: 
            dup_list = filtered_duplication
            run_slide = True
            use_filtered = True
        else:
            run_slide = False
    except NameError:
        dup_list = duplication
        run_slide = True 
    if run_slide: 
        print(p,calculate_position(p), dup_list[p])
        TradeMarginTable(prs,client_manuf_segment_dfs_new,top_competitor_manuf_segment_dfs_new,dup_list[p],position = calculate_position(p,use_filtered),Inscope="Segment")
        p+=1
if len(subsegments)!=0: 
    try:
        if 9 in filtered_index and 'Trade Margin Table By Subsegment' in filtered_section_names: 
            dup_list = filtered_duplication
            run_slide = True
            use_filtered = True
        else:
            run_slide = False
    except NameError:
        dup_list = duplication
        run_slide = True 
    if run_slide: 
        TradeMarginTable(prs,client_manuf_subsegment_dfs_new,top_competitor_manuf_subsegment_dfs_new,dup_list[p],position = calculate_position(p,use_filtered),Inscope="SubSegment")
        p+=1   
if len(subcategories)!=0:
    try:
        if 9 in filtered_index and 'Trade Margin Table By Subcategory' in filtered_section_names: 
            dup_list = filtered_duplication
            run_slide = True
            use_filtered = True
        else:
            run_slide = False
    except NameError:
        dup_list = duplication
        run_slide = True 
    if run_slide:
        TradeMarginTable(prs,client_manuf_subcategory_dfs_new,top_competitor_manuf_subcategory_dfs_new,dup_list[p],position = calculate_position(p,use_filtered),Inscope="SubCategory")
        p+=1


1 5 4


In [ ]:
outputPath=os.getcwd() +f"\\Financials_Output_{datetime.today().strftime("%d-%m")}.pptx"
prs.save(outputPath)
# app = win32.Dispatch("PowerPoint.Application")/
# presentation = app.Presentations.Open(outputPath)

In [ ]:
final=os.getcwd() +f"\\Financials_Output_{datetime.today().strftime("%d-%m")}.pptx"

In [ ]:
# Example usage
pptx_path = outputPath  # Replace with the actual path to your PPTX file
output_pptx_path=final
# open_chart_data_in_excel(pptx_path,output_pptx_path)
